In [1]:
import graphlab
from graphlab import SFrame

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days
/opt/conda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [4]:
loans = graphlab.SFrame('sub_lending-club-data.csv')

/opt/conda/lib/python2.7/site-packages/requests/packages/urllib3/connection.py:266: SubjectAltNameWarning: Certificate for beta.graphlab.com has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SubjectAltNameWarning
[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1500728060.log


This non-commercial license of GraphLab Create for academic use is assigned to mukesh.mithrakumar@jacks.sdstate.edu and will expire on June 17, 2018.


Finished parsing file /home/jovyan/work/Course3/Week4/sub_lending-club-data.csv

Parsing completed. Parsed 100 lines in 1.59167 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,int,int,int,str,float,float,str,str,str,str,str,int,str,str,str,str,str,str,str,str,str,str,float,int,str,int,int,int,int,int,int,float,int,str,float,float,float,float,float,float,float,float,float,str,float,str,str,int,str,int,int,str,int,int,int,int,float,int,int,int,int,float,str,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/jovyan/work/Course3/Week4/sub_lending-club-data.csv

Parsing completed. Parsed 37892 lines in 0.951381 secs.

In [8]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.remove_column('bad_loans')

In [9]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

In [10]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(percentage, seed = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
print "Total number of loans in our new dataset :", len(loans_data)

Percentage of safe loans                 : 0.500912778905
Percentage of risky loans                : 0.499087221095
Total number of loans in our new dataset : 9860


In [11]:
#Transform categorical data into binary features

loans_data = risky_loans.append(safe_loans)
for feature in features:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})    
    loans_data_unpacked = loans_data_one_hot_encoded.unpack(column_name_prefix=feature)
    
    # Change None's to 0's
    for column in loans_data_unpacked.column_names():
        loans_data_unpacked[column] = loans_data_unpacked[column].fillna(0)

    loans_data.remove_column(feature)
    loans_data.add_columns(loans_data_unpacked)

In [13]:
features = loans_data.column_names()
features.remove('safe_loans')  # Remove the response variable

In [14]:
train_data, validation_set = loans_data.random_split(.8, seed=1)

In [25]:
def reached_minimum_node_size(data, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    if len(data) <= min_node_size:
        return True

In [21]:
def error_reduction(error_before_split, error_after_split):
    # Return the error before the split minus the error after the split.
    diff = error_before_split - error_after_split
    return diff

In [22]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0    
    # Count the number of 1's (safe loans)
    sum_safe_loans = (labels_in_node == +1).sum()
    # Count the number of -1's (risky loans)
    sum_risky_loans = (labels_in_node == -1).sum()
            
    # Return the number of mistakes that the majority classifier makes.
    return sum_risky_loans if sum_safe_loans > sum_risky_loans else sum_safe_loans

In [23]:
def best_splitting_feature(data, features, target):
    
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:      
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]       
        # The right split will have all data points where the feature value is 1
        right_split = data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        left_mistakes = intermediate_node_num_mistakes(left_split[target])       
        # Calculate the number of misclassified examples in the right split.
        right_mistakes = intermediate_node_num_mistakes(right_split[target])    
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        error = (left_mistakes + right_mistakes) / num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        if error < best_error:
            best_feature = feature
            best_error = error
    return best_feature # Return the best feature we found

In [24]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True}
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = +1
    else:
        leaf['prediction'] = -1
        
    # Return the leaf node        
    return leaf

In [26]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    
    
    # Stopping condition 1: All nodes are of the same type.
    if intermediate_node_num_mistakes(target_values) == 0:
        print "Stopping condition 1 reached. All data points have the same target value."                
        return create_leaf(target_values)
    
    # Stopping condition 2: No more features to split on.
    if remaining_features == []:
        print "Stopping condition 2 reached. No remaining features."                
        return create_leaf(target_values)    
    
    # Early stopping condition 1: Reached max depth limit.
    if current_depth >= max_depth:
        print "Early stopping condition 1 reached. Reached maximum depth."
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if reached_minimum_node_size(data, min_node_size):
        print "Early stopping condition 2 reached. Reached minimum node size."
        return create_leaf(target_values)
    
    # Find the best splitting feature
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes = intermediate_node_num_mistakes(left_split[target])
    right_mistakes = intermediate_node_num_mistakes(right_split[target])
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if error_reduction(error_before_split, error_after_split) <= min_error_reduction:
        print "Early stopping condition 3 reached. Minimum error reduction."
        return create_leaf(target_values)
    
    remaining_features.remove(splitting_feature)
    print "Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split))
    
    
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        
    
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [27]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

In [28]:
#TEST

small_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 2, 
                                        min_node_size = 10, min_error_reduction=0.0)
if count_nodes(small_decision_tree) == 7:
    print 'Test passed!'
else:
    print 'Test failed... try again!'
    print 'Number of nodes found                :', count_nodes(small_decision_tree)
    print 'Number of nodes that should be there : 7' 

--------------------------------------------------------------------
Subtree, depth = 0 (7906 data points).
Split on feature term.36 months. (2299, 5607)
--------------------------------------------------------------------
Subtree, depth = 1 (2299 data points).
Split on feature grade.A. (2247, 52)
--------------------------------------------------------------------
Subtree, depth = 2 (2247 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (52 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (5607 data points).
Split on feature grade.E. (5398, 209)
--------------------------------------------------------------------
Subtree, depth = 2 (5398 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------

In [29]:
my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (7906 data points).
Split on feature term.36 months. (2299, 5607)
--------------------------------------------------------------------
Subtree, depth = 1 (2299 data points).
Split on feature grade.A. (2247, 52)
--------------------------------------------------------------------
Subtree, depth = 2 (2247 data points).
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (52 data points).
Early stopping condition 2 reached. Reached minimum node size.
--------------------------------------------------------------------
Subtree, depth = 1 (5607 data points).
Split on feature grade.E. (5398, 209)
--------------------------------------------------------------------
Subtree, depth = 2 (5398 data points).
Split on feature grade.D. (4691, 707)
--------------------------------------------------------------------
Su

In [30]:
my_decision_tree_old = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (7906 data points).
Split on feature term.36 months. (2299, 5607)
--------------------------------------------------------------------
Subtree, depth = 1 (2299 data points).
Split on feature grade.A. (2247, 52)
--------------------------------------------------------------------
Subtree, depth = 2 (2247 data points).
Split on feature grade.B. (1766, 481)
--------------------------------------------------------------------
Subtree, depth = 3 (1766 data points).
Split on feature grade.C. (1285, 481)
--------------------------------------------------------------------
Subtree, depth = 4 (1285 data points).
Split on feature grade.D. (794, 491)
--------------------------------------------------------------------
Subtree, depth = 5 (794 data points).
Split on feature grade.E. (305, 489)
--------------------------------------------------------------------
Subtree, depth = 6 (305 data points).
Early stoppin

In [31]:
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [32]:
validation_set[0]

{'emp_length.1 year': 0,
 'emp_length.10+ years': 0,
 'emp_length.2 years': 1,
 'emp_length.3 years': 0,
 'emp_length.4 years': 0,
 'emp_length.5 years': 0,
 'emp_length.6 years': 0,
 'emp_length.7 years': 0,
 'emp_length.8 years': 0,
 'emp_length.9 years': 0,
 'emp_length.< 1 year': 0,
 'emp_length.n/a': 0,
 'grade.A': 0,
 'grade.B': 0,
 'grade.C': 0,
 'grade.D': 1,
 'grade.E': 0,
 'grade.F': 0,
 'grade.G': 0,
 'home_ownership.MORTGAGE': 0,
 'home_ownership.OTHER': 0,
 'home_ownership.OWN': 0,
 'home_ownership.RENT': 1,
 'safe_loans': -1,
 'term.36 months': 0,
 'term.60 months': 1}

In [33]:
print 'Predicted class: %s ' % classify(my_decision_tree_new, validation_set[0])

Predicted class: -1 


In [34]:
classify(my_decision_tree_new, validation_set[0], annotate = True)

Split on term.36 months = 0
Split on grade.A = 0
At leaf, predicting -1


-1

In [35]:
classify(my_decision_tree_old, validation_set[0], annotate = True)

Split on term.36 months = 0
Split on grade.A = 0
Split on grade.B = 0
Split on grade.C = 0
Split on grade.D = 1
Split on grade.E = 0
At leaf, predicting -1


-1

In [36]:
def evaluate_classification_error(tree, data, target):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x))
    num_data = float(len(data))
    # Once you've made the predictions, calculate the classification error and return it
    mistakes = (prediction != data[target]).sum()
    return mistakes / num_data

In [37]:
evaluate_classification_error(my_decision_tree_new, validation_set, target)

0.33725690890481064

In [38]:
evaluate_classification_error(my_decision_tree_old, validation_set, target)

0.33776867963152507

In [41]:
model_1 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 2, 
                                min_node_size = 0, min_error_reduction=-1)
model_2 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)
model_3 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 14, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (7906 data points).
Split on feature term.36 months. (2299, 5607)
--------------------------------------------------------------------
Subtree, depth = 1 (2299 data points).
Split on feature grade.A. (2247, 52)
--------------------------------------------------------------------
Subtree, depth = 2 (2247 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (52 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (5607 data points).
Split on feature grade.E. (5398, 209)
--------------------------------------------------------------------
Subtree, depth = 2 (5398 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------

In [42]:
print "Training data, classification error (model 1):", evaluate_classification_error(model_1, train_data, target)
print "Training data, classification error (model 2):", evaluate_classification_error(model_2, train_data, target)
print "Training data, classification error (model 3):", evaluate_classification_error(model_3, train_data, target)

Training data, classification error (model 1): 0.357955982798
Training data, classification error (model 2): 0.341512775108
Training data, classification error (model 3): 0.332279281558


In [43]:
print "Validation data, classification error (model 1):", evaluate_classification_error(model_1, validation_set, target)
print "Validation data, classification error (model 2):", evaluate_classification_error(model_2, validation_set, target)
print "Validation data, classification error (model 3):", evaluate_classification_error(model_3, validation_set, target)

Validation data, classification error (model 1): 0.362845445241
Validation data, classification error (model 2): 0.337768679632
Validation data, classification error (model 3): 0.348004094166


In [44]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

In [45]:
nodes_m1 = count_leaves(model_1)
nodes_m2 = count_leaves(model_2)
nodes_m3 = count_leaves(model_3)
print "Nodes in model 1: ", nodes_m1
print "Nodes in model 2: ", nodes_m2
print "Nodes in model 3: ", nodes_m3

Nodes in model 1:  4
Nodes in model 2:  52
Nodes in model 3:  393


In [46]:
model_4 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)
model_5 =decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=0)
model_6 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=5)

--------------------------------------------------------------------
Subtree, depth = 0 (7906 data points).
Split on feature term.36 months. (2299, 5607)
--------------------------------------------------------------------
Subtree, depth = 1 (2299 data points).
Split on feature grade.A. (2247, 52)
--------------------------------------------------------------------
Subtree, depth = 2 (2247 data points).
Split on feature grade.B. (1766, 481)
--------------------------------------------------------------------
Subtree, depth = 3 (1766 data points).
Split on feature grade.C. (1285, 481)
--------------------------------------------------------------------
Subtree, depth = 4 (1285 data points).
Split on feature grade.D. (794, 491)
--------------------------------------------------------------------
Subtree, depth = 5 (794 data points).
Split on feature grade.E. (305, 489)
--------------------------------------------------------------------
Subtree, depth = 6 (305 data points).
Early stoppin

In [47]:
print "Validation data, classification error (model 4):", evaluate_classification_error(model_4, validation_set, target)
print "Validation data, classification error (model 5):", evaluate_classification_error(model_5, validation_set, target)
print "Validation data, classification error (model 6):", evaluate_classification_error(model_6, validation_set, target)

Validation data, classification error (model 4): 0.337768679632
Validation data, classification error (model 5): 0.335209825998
Validation data, classification error (model 6): 0.522517911975


In [48]:
nodes_m4 = count_leaves(model_4)
nodes_m5 = count_leaves(model_5)
nodes_m6 = count_leaves(model_6)
print "Nodes in model 4: ", nodes_m4
print "Nodes in model 5: ", nodes_m5
print "Nodes in model 6: ", nodes_m6

Nodes in model 4:  52
Nodes in model 5:  25
Nodes in model 6:  1


In [49]:
model_7 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)
model_8 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 2000, min_error_reduction=-1)
model_9 =decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 50000, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (7906 data points).
Split on feature term.36 months. (2299, 5607)
--------------------------------------------------------------------
Subtree, depth = 1 (2299 data points).
Split on feature grade.A. (2247, 52)
--------------------------------------------------------------------
Subtree, depth = 2 (2247 data points).
Split on feature grade.B. (1766, 481)
--------------------------------------------------------------------
Subtree, depth = 3 (1766 data points).
Split on feature grade.C. (1285, 481)
--------------------------------------------------------------------
Subtree, depth = 4 (1285 data points).
Split on feature grade.D. (794, 491)
--------------------------------------------------------------------
Subtree, depth = 5 (794 data points).
Split on feature grade.E. (305, 489)
--------------------------------------------------------------------
Subtree, depth = 6 (305 data points).
Early stoppin

In [50]:
print "Validation data, classification error (model 7):", evaluate_classification_error(model_7, validation_set, target)
print "Validation data, classification error (model 8):", evaluate_classification_error(model_8, validation_set, target)
print "Validation data, classification error (model 9):", evaluate_classification_error(model_9, validation_set, target)

Validation data, classification error (model 7): 0.337768679632
Validation data, classification error (model 8): 0.335721596725
Validation data, classification error (model 9): 0.522517911975


In [51]:
nodes_m7 = count_leaves(model_7)
nodes_m8 = count_leaves(model_8)
nodes_m9 = count_leaves(model_9)
print "Nodes in model 7: ", nodes_m7
print "Nodes in model 8: ", nodes_m8
print "Nodes in model 9: ", nodes_m9

Nodes in model 7:  52
Nodes in model 8:  9
Nodes in model 9:  1
